# How to migrate chains to LCEL

:::info Prerequisites

This guide assumes familiarity with the following concepts:
- [LangChain Expression Language](/docs/concepts#langchain-expression-language-lcel)

:::

LCEL is designed to streamline the process of building useful apps with LLMs and combining related components. It does this by providing:

1. **A unified interface**: Every LCEL object implements the `Runnable` interface, which defines a common set of invocation methods (`invoke`, `batch`, `stream`, `ainvoke`, ...). This makes it possible to also automatically and consistently support useful operations like streaming of intermediate steps and batching, since every chain composed of LCEL objects is itself an LCEL object.
2. **Composition primitives**: LCEL provides a number of primitives that make it easy to compose chains, parallelize components, add fallbacks, dynamically configure chain internals, and more.

LangChain maintains a number of legacy abstractions. Many of these can be reimplemented via short combinations of LCEL primitives. Doing so confers some advantages:

- The resulting chains typically implement the full `Runnable` interface, including streaming and asynchronous support where appropriate;
- The chains may be more easily extended or modified;
- The parameters of the chain are typically surfaced for easier customization (e.g., prompts) over previous versions, which tended to be subclasses and had opaque parameters and internals.

The LCEL implementations can be slightly more verbose, but there are significant benefits in transparency and customizability.

In this guide we review LCEL implementations of common legacy abstractions. Where appropriate, we link out to separate guides with more detail.

In [ ]:
%pip install --upgrade --quiet langchain-core langchain langchain-openai

## LLMChain

[`LLMChain`](https://api.python.langchain.com/en/latest/chains/langchain.chains.llm.LLMChain.html) combined a prompt template, LLM, and output parser into a class.

Some advantages of switching to LCEL to replace it are:

- Clarity around contents and parameters. The legacy `LLMChain` contains a default output parser and other options.
- Easier streaming. `LLMChain` only supports streaming via callbacks.
- Easier access to raw model outputs if desired. `LLMChain` only exposes these via a parameter or via callback.

import { ColumnContainer, Column } from "@theme/Columns";

<ColumnContainer>

<Column>

#### Legacy


In [2]:
from langchain.chains import LLMChain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [("user", "Tell me a {adjective} joke")],
)

chain = LLMChain(llm=ChatOpenAI(), prompt=prompt)

chain("funny")

/Users/jacoblee/.pyenv/versions/3.10.5/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
/Users/jacoblee/.pyenv/versions/3.10.5/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{'adjective': 'funny',
 'text': "Why couldn't the bicycle stand up by itself?\n\nBecause it was two-tired!"}


</Column>

<Column>

#### LCEL



In [3]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [("user", "Tell me a {adjective} joke")],
)

chain = prompt | ChatOpenAI() | StrOutputParser()

chain.invoke({ "adjective": "funny" })

'Why did the scarecrow win an award? Because he was outstanding in his field!'


</Column>
</ColumnContainer>

Note that `LLMChain` by default returns a `dict` containing both the input and the output. If this behavior is desired, we can replicate it using another LCEL primitive, [`RunnablePassthrough`](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.passthrough.RunnablePassthrough.html):

In [4]:
from langchain_core.runnables import RunnablePassthrough

outer_chain = RunnablePassthrough().assign(text=chain)

outer_chain.invoke({ "adjective": "funny" })

{'adjective': 'funny',
 'text': 'Why was the math book sad?\n\nBecause it had too many problems!'}

See [this tutorial](/docs/tutorials/llm_chain) for more detail on building with prompt templates, LLMs, and output parsers.

## ConversationChain

[`ConversationChain`](https://api.python.langchain.com/en/latest/chains/langchain.chains.conversation.base.ConversationChain.html) incorporates a memory of previous messages to sustain a stateful conversation.

Some advantages of switching to LCEL to replace it are:

- Innate support for threads/separate sessions. To make this work with `ConversationChain`, you'd need to instantiate a separate memory class outside the chain.
- More explicit parameters. `ConversationChain` contains a hidden default prompt.
- Streaming support. `ConversationChain` only supports streaming via callbacks.

`RunnableWithMessageHistory` implements sessions via configuration parameters. It should be instantiated with a callable that returns a [chat message history](https://api.python.langchain.com/en/latest/chat_history/langchain_core.chat_history.BaseChatMessageHistory.html). By default, it expects this function to take a single argument `session_id`.

<ColumnContainer>
<Column>

#### Legacy


In [5]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI

memory = ConversationBufferMemory()
chain = ConversationChain(
    llm=ChatOpenAI(),
    memory=memory
)

chain({"input": "how are you?"})

{'input': 'how are you?',
 'history': '',
 'response': "I'm doing well, thank you for asking! I've been busy analyzing data and learning new information. How about you, how are you doing today?"}

</Column>

<Column>

#### LCEL



In [6]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template(
    "You are a pirate. Answer the following questions as best you can:\n{input}"
)
history = InMemoryChatMessageHistory()
chain = RunnableWithMessageHistory(
    prompt | ChatOpenAI(),
    lambda x: history
)

chain.invoke(
    {"input": "how are you?"},
    config={"configurable": {"session_id": "42"}}
)

AIMessage(content="Arrr matey, I be sailin' the high seas and feelin' like a true swashbuckler! Aye, I be doin' just fine, thank ye for askin'. What be ye needin' from this ol' pirate?", response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 31, 'total_tokens': 86}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-01f30388-cc24-4977-9f64-73f5d38a60c8-0', usage_metadata={'input_tokens': 31, 'output_tokens': 55, 'total_tokens': 86})


</Column>
</ColumnContainer>

The above example uses the same `history` for all sessions. The example below shows how to use a different chat history for each session.

In [7]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


chain = RunnableWithMessageHistory(prompt | ChatOpenAI(), get_session_history)

chain.invoke("Hello!", config={"configurable": {"session_id": "abc123"}})

AIMessage(content="Arrr matey! What be ye wantin' from this ole pirate?", response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 29, 'total_tokens': 45}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0db337ee-c163-42e7-b0f0-cc8668551f03-0', usage_metadata={'input_tokens': 29, 'output_tokens': 16, 'total_tokens': 45})

See [this tutorial](/docs/tutorials/chatbot) for a more end-to-end guide on building with [`RunnableWithMessageHistory`](https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html).

## ConversationRetrievalChain

The [`ConversationalRetrievalChain`](https://api.python.langchain.com/en/latest/chains/langchain.chains.conversational_retrieval.base.ConversationalRetrievalChain.html) was an all-in one way that combined retrieval-augmented generation with chat history, allowing you to "chat with" your documents.

Some advantages of switching to LCEL to replace it are: